In [67]:
!pip install py_arkworks_bls12381

from py_arkworks_bls12381 import G1Point, G2Point, GT, Scalar
from sympy import symbols, div, Poly, mod_inverse
import random
from random import randint
from numpy.polynomial import polynomial as P
import numpy as np

In [68]:
# Initialize the Elliptic Curve
g = G1Point()
h = G2Point()
gt_gen = GT()

g_identity = G1Point.identity()
h_identity = G2Point.identity()

field_order = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
subgroup_order = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001

In [69]:
# Tools

# Check if the polynomial's coefs are integer
def assert_integer_polynomial(f):
    assert all(int(coeff) == coeff for coeff in f)

# evaluate polynomial f on point s
# f = [3, 2, 1] <-> 3*x^2 + 2*x + 1
def polynomial_eval(f, s):
    sum = f[-1]
    for i in range(1, len(f)):
        sum += f[-(i + 1)] * s**i
    return sum

# f is a list of coeffs s.t. f = [3, 2] <-> 3x + 2
def trapdoor_polynomial_eval(group_identity, f, powers_of_tau):
    """Evaluate polynomial f on tau in group"""
    c_f = group_identity

    for i in range(len(f)):
        f_i = int(f[-(i + 1)])
        if f_i > 0:
            c_f += powers_of_tau[i] * Scalar(f_i)
        else:
            c_f -= powers_of_tau[i] * Scalar(-f_i)

    return c_f

def vanishing_poly(points, mod):
    """Returns vanishing polynomial A_I(X) = Π(X - i) for i in points"""
    x = symbols('x')
    A = Poly(1, x).set_modulus(mod)
    for i in range(len(points)):
        A *= Poly([1, -points[i]], x).set_modulus(mod)
    return A

def lagrange_polynomial(points, s, mod):
    """Returns polynomial L(x) s.t. L = Π(X - i) / (s - i) for every i != s in I"""
    x = symbols('x')
    numerator = Poly(1, x).set_modulus(mod)
    denominator = Poly(1, x).set_modulus(mod)
    for point in points:
        if point != s:
            numerator *= Poly([1, -point], x).set_modulus(mod)
            denominator *= Poly([s - point], x).set_modulus(mod)
    Q, _ = div(numerator, denominator)
    return Q

def RI(points, values, mod):
    """Returns polynomial RI(x) s.t. RI(point_i) = value_i for all points in I"""
    x = symbols('x')
    r_poly = Poly([0], x).set_modulus(mod)
    for point, value in zip(points, values):
        V = Poly([value], x).set_modulus(mod)

        r_poly += V.set_modulus(mod) * lagrange_polynomial(points, point, mod)
    return r_poly

In [70]:
class Trusted_Setup:
    def __init__(self, d):
        self.d = d

    # input d, the maximum degree of a polynomial we will commit to
    # returns (ck, vk)
    # ck is the commitment key which is the list: ([1]_1, [tau]_1, ..., [tau ^ d]_1)
    # vk is the verification key which is a pair: ([1]_2, [tau]_2)
    def generate_public_parameters(self):
        keys = {}
        keys["ck"] = [];
        keys["vk"] = [];

        tau = Scalar(randint(0, 2**255 - 1))

        coef = Scalar(1)
        for _ in range(self.d):
            keys["ck"].append(g * coef)
            coef *= tau

        coef = Scalar(1)
        for _ in range(self.d):
            keys["vk"].append(h * coef)
            coef *= tau

        return keys

In [71]:
class Prover:
    def __init__(self, ck, f):
        self.ck = ck
        self.f = f

    # input f is the polynomial we are commiting to
    # input ck is a list (powers of tau from 1 to d). The commitment key generated by the trusted setup
    # returns c_f, the commited point on G1
    def commit(self):
        assert len(self.f) <= len(self.ck), "Polynomial degree exceeds commitment key size"

        assert_integer_polynomial(self.f)

        return trapdoor_polynomial_eval(g_identity, self.f, self.ck)

    def quotient_poly(self, points):
        values = [int(polynomial_eval(self.f, s)) % subgroup_order for s in points]

        x = symbols('x')
        numerator_poly = Poly(self.f.copy(), x).set_modulus(subgroup_order) - RI(points, values, subgroup_order)

        denominator_poly = vanishing_poly(points, subgroup_order)

        q, _ = div(numerator_poly, denominator_poly)

        return q

    # "points" is a list of points on G1
    def eval(self, points):
        q = self.quotient_poly(points)

        return trapdoor_polynomial_eval(g_identity, q.all_coeffs(), self.ck)

In [72]:
class Verifier:
    def __init__(self, vk):
        self.vk = vk

    def verify(self, polynomial_commitment, points, values, quotient_polynomial_commitment):
        assert(self.vk[0] == h)

        r_poly = RI(points, values, subgroup_order)
        r_poly_on_tau = trapdoor_polynomial_eval(g_identity, r_poly.all_coeffs(), ck)

        a_poly = vanishing_poly(points, subgroup_order)
        a_poly_on_tau = trapdoor_polynomial_eval(h_identity, a_poly.all_coeffs(), vk)

        lhs = GT.pairing(polynomial_commitment - r_poly_on_tau, self.vk[0])
        rhs = GT.pairing(quotient_polynomial_commitment, a_poly_on_tau)

        return lhs == rhs

In [73]:
# Unit Tests

max_d = 100
trusted_setup = Trusted_Setup(max_d)
pp = trusted_setup.generate_public_parameters()
ck = pp["ck"]
vk = pp["vk"]

def single_point_vanishing_polynomial_check():
    points = [-1]
    poly = vanishing_poly(points, subgroup_order)

    correct_answer = [1, 1]
    assert(poly.all_coeffs() == correct_answer)

def two_points_vanishing_polynomial_check():
    points = [1, 3]
    poly = vanishing_poly(points, subgroup_order)

    correct_answer = [1, -4, 3]
    assert(poly.all_coeffs() == correct_answer)

def single_point_small_polynomial_quotient_check():
    # 3x^2 + 2x + 1
    f = [3, 2, 1]

    prover = Prover(ck, f)
    points = [2]

    correct_answer = [3, 8]
    assert(prover.quotient_poly(points).all_coeffs() == correct_answer)

def two_points_small_polynomial_quotient_check_1():
    # 3x^2 - x + 2
    f = [3, -1, 2]

    prover = Prover(ck, f)
    points = [1, 2]

    correct_answer = [3]
    assert(prover.quotient_poly(points).all_coeffs() == correct_answer)

def two_points_small_polynomial_quotient_check_2():
    # 2x^2 + 1
    f = [2, 0, 1]

    prover = Prover(ck, f)
    points = [1, 3]

    correct_answer = [2]

    assert(prover.quotient_poly(points).all_coeffs() == correct_answer)

single_point_vanishing_polynomial_check()
two_points_vanishing_polynomial_check()

single_point_small_polynomial_quotient_check()
two_points_small_polynomial_quotient_check_1()
two_points_small_polynomial_quotient_check_2()

In [74]:
# End to End Tests: Single Points (Classic KZG)

max_d = 100
trusted_setup = Trusted_Setup(max_d)
pp = trusted_setup.generate_public_parameters()
ck = pp["ck"]
vk = pp["vk"]

def single_point_manual_polynomial_correct_commitment_1():
    # x + 1
    f = [1, 1]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()
    points = [2]
    correct_values = [polynomial_eval(f, points[0])]
    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, correct_values, quotient_polynomial_commitment)

def single_point_manual_polynomial_correct_commitment_2():
    # 3x^2 + 2x + 1
    f = [1, 2, 3]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()
    points = [5]
    correct_values = [polynomial_eval(f, points[0])]
    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, correct_values, quotient_polynomial_commitment)

def single_point_small_random_polynomial_wrong_evaluation_commitment():
    f_deg = 7
    max_coef = 7

    f = [randint(0, max_coef) for _ in range(f_deg)]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()
    points = [randint(0, 100)]
    correct_values = [int(polynomial_eval(f, points[0]))]
    wrong_values = [correct_values[0] + 1]

    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, wrong_values, quotient_polynomial_commitment)

def single_point_small_random_polynomial_correct_commitment():
    f_deg = 100
    max_coef = 100

    f = [randint(0, max_coef) for _ in range(f_deg)]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()
    points = [randint(0, 100)]
    correct_values = [int(polynomial_eval(f, points[0]))]
    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, correct_values, quotient_polynomial_commitment)

assert(single_point_manual_polynomial_correct_commitment_1() == True)
assert(single_point_manual_polynomial_correct_commitment_2() == True)
assert(single_point_small_random_polynomial_correct_commitment() == True)
assert(single_point_small_random_polynomial_wrong_evaluation_commitment() == False)

In [75]:
# End to End Tests: Multiple Points

max_d = 100
trusted_setup = Trusted_Setup(max_d)
pp = trusted_setup.generate_public_parameters()
ck = pp["ck"]
vk = pp["vk"]

def multiple_point_manual_polynomial_correct_commitment_1():
    # x + 1
    f = [1, 1]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()
    points = [1, 2]
    correct_values = [polynomial_eval(f, point) for point in points]
    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, correct_values,
                           quotient_polynomial_commitment)

def multiple_point_manual_polynomial_correct_commitment_2():
    # 3x^2 + 2x + 1
    f = [1, 2, 3]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()
    points = [5, 0, 1]
    correct_values = [polynomial_eval(f, point) for point in points]
    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, correct_values,
                           quotient_polynomial_commitment)

def multiple_point_small_random_polynomial_correct_commitment():
    points_count = 34

    f_deg = 100
    max_coef = 100

    f = [randint(0, max_coef) for _ in range(f_deg)]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()
    points = random.sample(range(0, 100), points_count)

    correct_values = [int(polynomial_eval(f, point)) for point in points]
    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, correct_values,
                           quotient_polynomial_commitment)

def multiple_point_small_random_polynomial_wrong_evaluation_commitment():
    points_count = 10

    f_deg = 7
    max_coef = 7

    f = [randint(0, max_coef) for _ in range(f_deg)]

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    polynomial_commitment = prover.commit()

    points = [randint(0, 100) for _ in range(points_count)]
    correct_values = [int(polynomial_eval(f, point)) for point in points]
    wrong_values = correct_values
    wrong_values[0] += 1

    quotient_polynomial_commitment = prover.eval(points)

    return verifier.verify(polynomial_commitment, points, wrong_values,
                           quotient_polynomial_commitment)

assert(multiple_point_manual_polynomial_correct_commitment_1() == True)
assert(multiple_point_manual_polynomial_correct_commitment_2() == True)
assert(multiple_point_small_random_polynomial_correct_commitment() == True)
assert(multiple_point_small_random_polynomial_wrong_evaluation_commitment() == False)